In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated \ufeffApril 20, 2020\r\n']

In [5]:
if len(data)>0:
    m = re.search(".*pdated (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
    data_date_str = m.group(1)
    data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
else:
    data_date = datetime.utcnow().strftime('%Y-%m-%d')
data_date

'2020-04-20'

### Extract data and add into Dataframe

In [6]:
dfs = pd.read_html(url)

In [7]:
if len(dfs) == 0 :
    raise Exception('There is no dataframes detected')

In [8]:
dfs

[                           Unnamed: 0_level_0  \
                             Age group (years)   
 0                                      Totals   
 1                    Race missing/unspecified   
 2                              Race specified   
 3             Among those with race specified   
 4            American Indian or Alaska Native   
 5                                       Asian   
 6                   Black or African American   
 7   Native Hawaiian or other Pacific Islander   
 8                                       White   
 9                              Multiple/other   
 10                                        NaN   
 11              Ethnicity missing/unspecified   
 12                        Ethnicity specified   
 13       Among those with ethnicity specified   
 14                            Hispanic/Latino   
 15                        Non-Hispanic/Latino   
 
                No. of cases (% of total)  \
                                     < 18   
 0      

In [9]:
df = dfs[0]
df

Unnamed: 0_level_0  \
                            Age group (years)   
0                                      Totals   
1                    Race missing/unspecified   
2                              Race specified   
3             Among those with race specified   
4            American Indian or Alaska Native   
5                                       Asian   
6                   Black or African American   
7   Native Hawaiian or other Pacific Islander   
8                                       White   
9                              Multiple/other   
10                                        NaN   
11              Ethnicity missing/unspecified   
12                        Ethnicity specified   
13       Among those with ethnicity specified   
14                            Hispanic/Latino   
15                        Non-Hispanic/Latino   

               No. of cases (% of total)  \
                                    < 18   
0                                  10160   
1                            7,587 (75%)   
2                            2,573 (25%)   
3        Among those with race specified   
4                              32 (1.2%)   
5                             124 (4.8%)   
6                            751 (29.2%)   
7                              10 (0.4%)   
8                          1,640 (63.7%)   
9                              16 (0.6%)   
10                                   NaN   
11                           7,534 (74%)   
12                           2,626 (26%)   
13  Among those with ethnicity specified   
14                             966 (37%)   
15                           1,660 (63%)   

                                          \
                                   18-44   
0                                 198588   
1                          140,413 (71%)   
2                           58,175 (29%)   
3        Among those with race specified   
4                             462 (0.8%)   
5                           3,762 (6.5%)   
6                         19,425 (33.4%)   
7                             253 (0.4%)   
8                         34,077 (58.6%)   
9                             196 (0.3%)   
10                                   NaN   
11                         140,594 (71%)   
12                          57,994 (29%)   
13  Among those with ethnicity specified   
14                          16,464 (28%)   
15                          41,530 (72%)   

                                          \
                                   45-64   
0                                 202331   
1                          133,741 (66%)   
2                           68,590 (34%)   
3        Among those with race specified   
4                             426 (0.6%)   
5                           4,213 (6.1%)   
6                         25,508 (37.2%)   
7                             225 (0.3%)   
8                         38,058 (55.5%)   
9                             160 (0.2%)   
10                                   NaN   
11                         137,226 (68%)   
12                          65,105 (32%)   
13  Among those with ethnicity specified   
14                          15,265 (23%)   
15                          49,840 (77%)   

                                          \
                                   65-74   
0                                  59177   
1                           33,100 (56%)   
2                           26,077 (44%)   
3        Among those with race specified   
4                             124 (0.5%)   
5                           1,330 (5.1%)   
6                          9,583 (36.7%)   
7                              73 (0.3%)   
8                         14,918 (57.2%)   
9                              49 (0.2%)   
10                                   NaN   
11                          36,168 (61%)   
12                          23,009 (39%)   
13  Among those with ethnicity specified   
14                           3,928 (17%)   
15               

In [10]:
csv_file='../data/cdc-us-cases-by-races-and-age-group.csv'
df.to_csv(csv_file, index=False)

In [11]:
# col0 = df.columns[0]
# df = df.set_index(col0).T
# df



In [12]:
# # remove non alphanumeric characters from the columns
# # also lowercase them
# df.columns = df.columns.map(lambda x: re.sub('\W+', '', str.lower(x)))

# # add column date to the front
# df.insert(0, 'date', data_date)


### Save to file (old data)


In [13]:
# from pathlib import Path
# csv_file = '../data/cdc-us-cases.csv'

# if Path(csv_file).exists():
#     # read out the old dat
#     old_df = pd.read_csv(csv_file)
#     df = df.append(old_df)
# df.drop_duplicates(subset=['date'], inplace=True)

In [14]:
# df.sort_values(by=['date'], inplace=True)
# df
# df.to_csv(csv_file, index=False)